<p style="float:right">
<img src="images/logos/cu.png" style="display:inline" />
<img src="images/logos/cires.png" style="display:inline" />
<img src="images/logos/nasa.png" style="display:inline" />
</p>

# Python, Jupyter & pandas: Module 5

## Inference and Visualization

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd


read in the csv file we saved from the last module

In [ ]:
monthly = pd.read_csv('monthly-extents.csv', index_col='date', parse_dates=True)

In [ ]:
monthly.head()

Look for a trend in the Northern Hemisphere June snowcover.

reset my DataFrame to months columns indexed by years.

In [ ]:
year_by_month = monthly.set_index([monthly.index.year, monthly.index.month]).unstack(1)

In [ ]:
year_by_month.head()

In [ ]:
june_anomalies = year_by_month['snowcover'][6] - year_by_month['snowcover'][6].mean()
june_anomalies = june_anomalies.dropna()

In [ ]:
with mpl.rc_context(rc={'figure.figsize': (15, 4)}):
    june_anomalies.plot(title='Northern Hemisphere Snow Cover Anomalies: June',kind='bar', color='r')
    

Compute a least squares linear fit

In [ ]:
slope, intercept = np.polyfit(june_anomalies.index.values, june_anomalies.values, 1)
fit_function = np.poly1d([ slope, intercept])
best_fit = fit_function(june_anomalies.index)

In [ ]:
with mpl.rc_context(rc={'figure.figsize': (15, 4)}):
    june_anomalies.plot(title='Northern Hemisphere Snow Cover Anomalies: June',kind='Bar', color='r')
    plt.plot(best_fit, color='b', linestyle='--')
